# 使用Apriori进行关联分析
Apriori算法
> 优点：易编码实现<br>
> 缺点：在大数据集上可能较慢<br>
> 适用数据类型：数值型数据或标称型数据

Apriori算法的一般过程：
> 1. 收集数据:使用任意方法。
> 2. 准备数据:任何数据类型都可以,因为我们只保存集合。
> 3. 分析数据:使用任意方法。
> 4. 训练算法:使用Apriori算法来找到频繁项集。
> 5. 测试算法:不需要测试过程。
> 6. 使用算法:用于发现频繁项集以及物品之间的关联规则。

## 1. 算法实现
### a. 生成候选项集
```
对数据集中的每条交易记录tran
    对每个候选项集can:
        检查一下can是否是tran的子集：
        如果是，则增加can的计数值
        对每个候选项集：
        如果其支持度不低于最小值，则保留该项集
        返回所有频繁项集列表
```

支持度定义：<br>
一个项集的支持度定义为在数据集中包含该项集的记录所占的比例，例如书中第一个例子中数据集总共有5条记录，那么{豆奶}的支持度就为4/5，因为它在4条记录里面都出现了

In [1]:
import numpy as np

In [2]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

def createC1(dataSet):
    """
    创建C1集合，将数据集中所有的元素分割成只含有一个元素的集合，
    然后放到C1中
    """
    
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    # frozenset：表示冻结的set集合，不可改变，可以当作字典的键值使用
    return list(map(frozenset, C1))
                
def scanD(D, Ck, minSupport):
    """
    计算候选数据集Ck在数据集D中的支持度，并返回支持度大于最小支持度的数据
    
    Parameters
    -----------
        D :           数据集
        Ck :          候选项集列表
        minSupport :  最小支持度
    Returns
    -----------
        retList :     支持度大于minSupport的集合
        supportData : 候选项集支持度数据
    """
    
    # ssCnt存放候选数据集Ck的频率，例如a->10, b->5, c->8 etc.
    ssCnt = {}
    for tid in D:
        for can in Ck:
            # s.issubset(t)   测试是否s中每个元素都在t中
            if can.issubset(tid):
                # 如果字典中没有当前项集，计数器置一
                if not can in ssCnt.keys():
                    ssCnt[can] = 1
                # 如果字典中有当前项集，计数器加一
                else:
                    ssCnt[can] += 1
        
    numItems = float(len(D))
    retList = []
    supportData = {}
    # 遍历所有候选集
    for key in ssCnt:
        # 计算每个候选集的频率
        support = ssCnt[key] / numItems
        # 大于最小支持度，则在频繁集列表中插入
        if support >= minSupport:
            retList.insert(0, key)
        # 记录当前集合的支持度数据
        supportData[key] = support
    return retList, supportData

In [3]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [4]:
C1 = createC1(dataSet)
print(C1)

[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]


In [5]:
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [6]:
L1, supportData0 = scanD(D, C1, 0.5)
print(L1); print(supportData0)

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]
{frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75}


## b. 实现完整的Apriori算法
伪代码如下：
```
当集合中项的个数大于0时：
    构建一个k个项组成的候选项集的列表
    检查数据以确认每个项集都是频繁的
    保留频繁项集并构建k+1个项组成的候选项集列表
```

In [7]:
def aprioriGen(Lk, k):
    """
    通过频繁项集Lk生成候选项集Ck
    例如：以{0}, {1}, {2}生成k=2的候选项集，则输出为{0,1}, {0,2}, {1,2}。
    
    Parameters
    -----------
        Lk : 频繁项集列表
        k : 返回的候选项集中元素的个数
    Returns
    -----------
        retList : 元素两两合并的数据集
    """
    
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            # k-2：现在想要从{0,1}, {0,2}, {1,2}构造出三元素项集，如果两两合并，那么会生成三个重复集合
            # 需要进一步去重操作，为了减少遍历列表的次数，只比较第一个元素并只对第一个元素相同的集合进行合并
            # 那么就可以直接得到{0, 1, 2}这个集合。因为Lk的元素是不重复的，所以直接合并速度较快
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            # 第一次两个列表元素为空，直接执行合并操作
            if L1 == L2:
                # | 集合的并操作
                retList.append(Lk[i] | Lk[j])
    return retList

def apriori(dataSet, minSupport=0.5):
    """
    Apriori算法实现
    
    Parameters
    -----------
        dataSet : 原始数据集
        minSupport : 最小支持度
    Returns
    -----------
        L : 频繁集的全集
        supportData : 支持度全集
    """
    # C1对原始数据集去重并排序
    C1 = createC1(dataSet)
    # 把数据集每行转换成set
    D = list(map(set, dataSet))
    # 基于原始数据集生成元素个数为1的频繁集
    L1, supportData = scanD(D, C1, minSupport)
    # 将第一个频繁集加入全集列表
    L = [L1]
    k = 2
    # 循环停止：当元素个数最多的频繁集已经达到上限，最后不能再生成元素更多的频繁集，停止循环
    while (len(L[k - 2]) > 0):
        # 由频繁集生成候选集，初始为2，后面依次累加
        Ck = aprioriGen(L[k - 2], k)
        # 通过前面生成的候选集生成频繁集
        Lk, supK = scanD(D, Ck, minSupport)
        # 更新支持度
        supportData.update(supK)
        # 更新频繁集全集列表
        L.append(Lk)
        k += 1
    return L, supportData

In [8]:
L, suppData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [9]:
aprioriGen(L[0], 2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

## 2. 从频繁项集挖掘关联规则
集合a到b的可信度定义为：
$$a->b = \frac{support(a | b)}{support(a)}$$

假设freqSet = frozenset([1,3])，conseq = [frozenset([1])]，那么frozenset([1])到frozenset([3])的可信度为：<br><br>
$$\frac{support(freqSet)}{support(conseq)}$$
反之，frozenset([3])到frozenset([1])的可信度为：
$$\frac{support(freqSet)}{support(freqSet - conseq)}$$

In [10]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    """
    对两个元素的频繁集计算可信度，例如：{1,2}/{1}或者{1,2}/{2}
    
    Parameters
    -----------
        freqSet : 频繁集中的元素，e.g.:frozenset({1, 3})
        H : 频繁集中的元素集合，e.g.:[frozenset({2, 3}), frozenset({3, 5})]
        supportData : 所有元素的支持度字典
        brl : 关联规则列表的数组
        minConf : 最小可信度
    Returns
    -----------
        prunedH : 记录可信度大于最小可信度的集合
    """
    prunedH = []
    # 假设freqSet = frozenset({1, 3})，H = [frozenset({1}), frozenset({3})]
    # 那么需要求出frozenset({1})->frozenset({3})和frozenset({3})->frozenset({1})的可信度
    for conseq in H:
        # 计算H中元素的可信度
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet-conseq, '--->', conseq, 'conf: ', conf)
            # 当前可信度较高，就扩充关联规则列表
            brl.append((freqSet - conseq, conseq, conf))
            # 加入输出结果
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    """
    生成关联规则
    
    Parameters
    -----------
        freqSet : 频繁集中的元素，e.g.:frozenset({1, 3})
        H : 频繁集中的元素集合，e.g.:[frozenset({2, 3}), frozenset({3, 5})]
        supportData : 所有元素的支持度字典
        brl : 关联规则列表的数组
        minConf : 最小可信度
    """
    # H[0]是freqSet中的第一个元素的frozenset，并且，H中所有元素的长度都一样均为m+1
    # 递归时，m从1开始增长到freqSet的产度为止，并不断地计算freqSet子集的可信度
    # 当H[0] = freqSet的时候停止递归
    m = len(H[0])
    if (len(freqSet) > m + 1):
        # 生成长度大于当前H元素长度的集合，例如：H = [frozenset([2]),frozenset([3]),frozenset([5])]
        # 那么就生成Hmp1 = [frozenset([2, 3]), forzenset([2, 5]), frozenset([3, 5])]
        Hmp1 = aprioriGen(H, m+1)
        # 获得所有可信度大于最小可信度的集合
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        # 计算可信度后，如果还有数据大于最小可信度的话继续递归，否则停止
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

def generateRules(L, supportData, minConf=0.7):
    """
    Returns
    -----------
        bigRuleList : 可信度规则列表，A->B + 可信度
    """
    bigRuleList = []
    # 对整个频繁项集子集遍历
    for i in range(1, len(L)):
        # 频繁项集每个组合的所有元素
        for freqSet in L[i]:
            # 假设freqSet = frozenset([1, 3]), H1 = [frozenset([1]), frozenset([3])]
            H1 = [frozenset([item]) for item in freqSet]
            # 对于freqSet中只含两个的元素计算可信度，两个以上的就可以生成关联规则了
            if i > 1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

In [11]:
L, supportData = apriori(dataSet, minSupport=0.5)
rules = generateRules(L, supportData, minConf=0.7)
rules

frozenset({5}) ---> frozenset({2}) conf:  1.0
frozenset({2}) ---> frozenset({5}) conf:  1.0
frozenset({1}) ---> frozenset({3}) conf:  1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]